
# LangChain with Google Generative AI and Pinecone

## Introduction

This project demonstrates my skills in utilizing advanced AI language models and integrations using LangChain, Google Generative AI (Gemini), and Pinecone. This notebook illustrates how to build a LangChain pipeline that interacts with both generative models and vector databases for efficient query embedding, text splitting, and similarity searches.

This notebook is designed to demonstrate how we can:
- Configure and interact with the `gemini-1.0-pro` model.
- Build a conversational AI agent using LangChain with the `gemini-1.0-pro` model.
- Create and customize prompt templates for specific AI tasks.
- Store and retrieve text embeddings using Pinecone.
- Perform similarity search over stored embeddings.

## Problem Statement

The goal of this notebook is to solve the problem of how to effectively manage large language models and vector embeddings, especially in contexts that require:
1. **Text Generation** - generating content, such as explaining concepts like "autoencoders" in AI.
2. **Embedding Search** - storing documents as embeddings and performing similarity searches.

This solution leverages LangChain to integrate various AI tools and frameworks in a modular and extensible way.

## Sections

- Setting up the environment.
- Running a basic query with the Gemini model.
- Chat interaction using the the model.
- Utilizing LangChain’s prompt templates for text generation.
- Performing embedding and similarity searches with Pinecone.
- Demonstrating LLMChain for integrating with Python tools.


### 1. Load environment variables

This step loads environment variables using `dotenv`. We need API keys for Google Generative AI and Pinecone, which are stored in a `.env` file. This setup ensures secure handling of sensitive information like API keys.


In [ ]:
# Load environment variables
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

### 2. Run basic query with Google Generative AI

Here, we use the `google.generativeai` module to configure the API key and run a basic query using the Gemini 1.0 model. This query asks the model to provide interesting information about the University at Buffalo.


In [ ]:
# Run basic query with OpenAI wrapper
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

### 3. Using LangChain to run a query with `gemini-1.5-pro`

In this example, we use LangChain’s integration with Google Generative AI to run a query where we ask the model to write a ballad about LangChain. This demonstrates how to interact with advanced language models through LangChain.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI  # type: ignore

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
llm.invoke("Write me a ballad about LangChain")

### 4. Using Gemini-1.5-pro for conversational AI

This example shows how we can use Gemini-1.5-pro in a conversational context. A chat system is defined where the model responds to user prompts. In this case, we ask the model to write a Python script that trains a neural network on simulated data.


In [ ]:
messages = [
    {"system", "You are an expert data scientist"},
    {"human", "Write a Python script that trains a neural network on simulated data "},
]
ai_msg = llm.invoke(messages)

print(ai_msg.content, end='\n')

### 6. Define and use a prompt template

Here, we define a `ChatPromptTemplate` where the system is an expert data scientist and the user can ask for an explanation of specific AI concepts. In this case, the model explains the concept of an "autoencoder".


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        {"system", "You are an expert data scientist with an expertise in building deep learning models."},
        {"human", "Explain the concept of {concept} in a couple of lines"},
    ]
)

chain = prompt | llm
chain.invoke({"concept": "autoencoder"})

### 7. Using LangChain's `LLMChain`

This block shows how to use an `LLMChain` to generate responses based on input. The chain runs the model to generate an explanation of an autoencoder.


In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("autoencoder"))

### 8. Sequential chain with two prompt templates

We create a `SimpleSequentialChain` to run two chains consecutively. The first chain generates an explanation of a concept, and the second chain builds on that output.


In [ ]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

explanation = overall_chain.run("autoencoder")
print(explanation)

### 9. Splitting text into chunks using `RecursiveCharacterTextSplitter`

To handle large texts, we split them into manageable chunks using LangChain’s `RecursiveCharacterTextSplitter`. This helps us store and process text more efficiently.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.create_documents([explanation])

### 10. Embedding text and setting up Pinecone for vector storage

We create embeddings for the text chunks and set up a Pinecone index for storing the embeddings. The embeddings are stored in a vector database, which can be used for fast similarity searches.


In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "knowledge-from-llms"
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

vector_store.add_documents(texts)

### 11. Similarity search with Pinecone

After storing the embeddings in Pinecone, we perform a similarity search to find the most relevant text based on a query. In this case, we ask, "What is magical about an autoencoder?" and retrieve the most similar document from our vector database.


In [ ]:
query = "What is magical about an autoencoder?"
query_embedding = embeddings.embed_query(query)
result = vector_store.similarity_search(query, 1)

print(result)

### 12. Running Python commands with LangChain and a REPL

In this section, we integrate a Python REPL tool with LangChain. This allows us to send commands to the model and, if necessary, execute Python code. For example, we ask the model to calculate the square of 7 and find the roots of a quadratic function.


In [ ]:
from langchain.chains import LLMChain
from langchain.experimental.tools import PythonREPLTool
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the LLM (Google Gemini Pro)
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")
python_repl = PythonREPLTool()

def run_agent(input_text):
    response = llm_chain.run(input_text)
    if "calculate" in input_text.lower():
        python_result = python_repl.run(response)
        return python_result
    return response

command = "Calculate the square of 7"
result = run_agent(command)
print(result)